<a href="https://colab.research.google.com/github/Yash-Kamtekar/Special-Topics-Assignment-2/blob/main/link_prediction_of_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Libraries

In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.1+cu113
     |████████████████████████████████| 7.9 MB 3.0 MB/s 
     |████████████████████████████████| 3.5 MB 2.9 MB/s 


Loading the libraries

In [2]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges
from torch_geometric.nn import GAE

Loading CiteSeer dataset

In [3]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
dataset.data

Processing...
Done!


Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])

In [4]:
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = None
data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327])

In [5]:
data = train_test_split_edges(data)

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


Encoder Initialization code

In [6]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)
    

Initialization of Auto Encoder

In [7]:
out_channels = 2
num_features = dataset.num_features
epochs = 100

model = GAE(GCNEncoder(num_features, out_channels))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [8]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    #if args.variational:
    #   loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)


In [9]:
def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)


In [10]:
for epoch in range(1, epochs + 1):
    loss = train()

    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    if epoch % 10 == 0:
      print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))


Epoch: 010, AUC: 0.6599, AP: 0.7058
Epoch: 020, AUC: 0.6582, AP: 0.7285
Epoch: 030, AUC: 0.6651, AP: 0.7374
Epoch: 040, AUC: 0.7426, AP: 0.7725
Epoch: 050, AUC: 0.7929, AP: 0.8057
Epoch: 060, AUC: 0.7993, AP: 0.8147
Epoch: 070, AUC: 0.8015, AP: 0.8177
Epoch: 080, AUC: 0.8037, AP: 0.8199
Epoch: 090, AUC: 0.8029, AP: 0.8185
Epoch: 100, AUC: 0.8006, AP: 0.8152


In [11]:
Z = model.encode(x, train_pos_edge_index)
Z

tensor([[-0.3541,  0.2276],
        [ 1.2069, -0.9004],
        [-0.8497,  0.5853],
        ...,
        [ 0.2748, -0.2017],
        [-0.9114,  0.6149],
        [-0.8698,  0.5949]], grad_fn=<AddBackward0>)